In [2]:
import tensorflow as tf 
from tensorflow.keras.layers import Flatten, Dense, Dropout
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
#limit memory

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 1*X GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=(1024*4))])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

In [4]:
# Loading the Xception model
base_model = tf.keras.applications.Xception(
    include_top=False,
    weights="Dataset/xception_weights_tf_dim_ordering_tf_kernels_notop.h5",
    input_shape=(256, 256, 3),
)

In [3]:
# Summary of the model
base_model.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 127, 127, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 127, 127, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 127, 127, 32) 0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [5]:
# Making layers of base model untrainable so that their weights won't change during training
for layer in base_model.layers:
    layer.trainable = False

In [6]:
# Defining the input and output layers of the base model
base_input = base_model.layers[0].input
base_output = base_model.layers[-1].output

In [7]:
# Adding the output layers to the base model
l1 = Flatten()(base_output)
l2 = Dense(512, activation='elu')(l1)
l3 = Dropout(0.2)(l2)
l4 = Dense(256, activation='elu')(l3)
l5 = Dropout(0.2)(l4)
l6 = Dense(12, activation='softmax')(l5)

model = tf.keras.Model(inputs=base_input, outputs=l6)

In [8]:
# Compiling the model i.e selecting the optimizer and loss function
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
model.compile(optimizer=optimizer, loss = 'categorical_crossentropy', metrics = ['acc'])

In [9]:
#Importing the data and augmenting it:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.15,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
        'Dataset/OG_Train',
        target_size=(256, 256),
        batch_size=69,
        class_mode='categorical')


valid_datagen = ImageDataGenerator(rescale=1/255)

valid_generator = valid_datagen.flow_from_directory(
        'Dataset/OG_Valid',
        target_size=(256, 256),
        batch_size=17,
        class_mode='categorical')

Found 1740 images belonging to 12 classes.
Found 443 images belonging to 12 classes.


In [10]:
# Setting up the callbacks

# checkpoint_cb will save the model which will perform the best on validation set
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint('Dataset/best_model.h5', save_best_only=True)

# early_stoping_cb will stop the model traning once it stops improving
early_stoping_cb = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

In [10]:
# training the model
# regularization
history = model.fit(
      train_generator,
      steps_per_epoch=25,
      validation_data=(valid_generator),
      callbacks=[checkpoint_cb, early_stoping_cb],  
      epochs=100,
      verbose=1)

Epoch 1/100
14/25 [===============>..............] - ETA: 1:20 - loss: 22.8340 - acc: 0.1568

KeyboardInterrupt: 

In [ ]:
# Performance of the model on training and validation sets
pd.DataFrame(history.history).plot()
plt.gca().set_ylim(0, 1)
plt.show()